In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils

Using TensorFlow backend.


1.讀取資料並顯示資訊，因為是照時間排序故日期欄位先去掉

In [3]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print(data.info())
print(data.head())

del data['Date']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2264 entries, 0 to 2263
Data columns (total 6 columns):
Date           2264 non-null object
Open Price     2264 non-null float64
Close Price    2264 non-null float64
High Price     2264 non-null float64
Low Price      2264 non-null float64
Volume         2264 non-null int64
dtypes: float64(4), int64(1), object(1)
memory usage: 106.2+ KB
None
          Date  Open Price  Close Price  High Price  Low Price      Volume
0  02-Jan-2009      902.99       931.80      934.73     899.35  4048270080
1  05-Jan-2009      929.17       927.45      936.63     919.53  5413910016
2  06-Jan-2009      931.17       934.70      943.85     927.28  5392620032
3  07-Jan-2009      927.45       906.65      927.45     902.37  4704940032
4  08-Jan-2009      905.73       909.73      910.00     896.81  4991549952


2.因為本身的欄位有點少，另外計算dir(漲或跌) dpre(前一天的變化幅度) dh(前一天的最高價變化幅度) dl(前一天的最低價變化幅度)

In [4]:
data['dir'] = data['Close Price'] - data['Open Price'] >= 0
data['dir'] = data['dir'].map({True: 1.0, False: 0.0})
data['dpre'] = data['Open Price'].diff().replace(np.nan, 0.0)
data['dh'] = data['High Price'].diff().replace(np.nan, 0.0)
data['dl'] = data['Low Price'].diff().replace(np.nan, 0.0)

3.計算和dir(漲或跌)的相關係數，取其最高3欄Open Price ,dh ,dl

In [5]:
print(data.corr()['dir'])

train_x = data[['Open Price' ,'dh', 'dl']]
train_y = data[['dir']]

Open Price    -0.033407
Close Price   -0.013586
High Price    -0.024132
Low Price     -0.021653
Volume        -0.023831
dir            1.000000
dpre           0.010067
dh             0.427708
dl             0.460147
Name: dir, dtype: float64


4.對test set做同樣處理

In [6]:
test['dir'] = test['Close Price'] - test['Open Price'] >= 0
test['dir'] = test['dir'].map({True: 1.0, False: 0.0})
test['dpre'] = test['Open Price'].diff().replace(np.nan, 0.0)
test['dh'] = test['High Price'].diff().replace(np.nan, 0.0)
test['dl'] = test['Low Price'].diff().replace(np.nan, 0.0)
test_y = test[['dir']]
test_x = test[['Open Price', 'dh', 'dl']]

5.套用logistic regression 計算在train和test的accuracy，solver挑了一下發現liblinear的結果比較好

In [7]:
lcf = LogisticRegression(solver='liblinear', random_state=1012)
lcf.fit(train_x, train_y)
print(accuracy_score(train_y, lcf.predict(train_x)))
print(accuracy_score(test_y, lcf.predict(test_x)))

0.7336572438162544
0.7301587301587301


訓練集測試集都很平均約73%

6.套用svc 計算在train和test的accuracy，設定max_iter=500 kernel='rbf'再test set的表現比較好

In [8]:
scf = SVC(random_state=1012, max_iter=500, kernel='rbf')
scf.fit(train_x, train_y)
print(accuracy_score(train_y, scf.predict(train_x)))
print(accuracy_score(test_y, scf.predict(test_x)))

0.8957597173144877
0.5436507936507936


訓練集約有9成的準度，不過測試集就掉下去了

7.套用nn 計算在train和test的accuracy (先把資料轉到ndaray)

In [9]:
train_y = np_utils.to_categorical(np.array(data[['dir']]))
train_x = np.array(train_x)
test_y = np_utils.to_categorical(np.array(test[['dir']]))
test_x = np.array(test_x)

model = Sequential()

model.add(Dense(units=8, input_dim=3, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=4, input_dim=6, kernel_initializer='normal', activation='sigmoid'))
model.add(Dense(units=2, kernel_initializer='normal', activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
train_history = model.fit(x=train_x, y=train_y, validation_split=0.1, epochs=20, batch_size=5, verbose=2)

scores = model.evaluate(train_x, train_y)
print(scores[1]*100.0)
scores = model.evaluate(test_x, test_y)
print(scores[1]*100.0)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 2037 samples, validate on 227 samples
Epoch 1/20
 - 0s - loss: 0.6714 - acc: 0.5498 - val_loss: 0.6631 - val_acc: 0.5683
Epoch 2/20
 - 0s - loss: 0.6093 - acc: 0.6917 - val_loss: 0.6159 - val_acc: 0.6476
Epoch 3/20
 - 0s - loss: 0.5879 - acc: 0.7212 - val_loss: 0.6121 - val_acc: 0.6388
Epoch 4/20
 - 0s - loss: 0.5721 - acc: 0.7270 - val_loss: 0.7077 - val_acc: 0.5903
Epoch 5/20
 - 0s - loss: 0.5643 - acc: 0.7285 - val_loss: 0.6088 - val_acc: 0.6520
Epoch 6/20
 - 0s - loss: 0.5617 - acc: 0.7236 - val_loss: 0.6125 - val_acc: 0.6388
Epoch 7/20
 - 0s - loss: 0.5556 - acc: 0.7300 - val_loss: 0.6946 - val_acc: 0.5771
Epoch 8/20
 - 0s - loss: 0.5517 - acc: 0.7280 - val_loss: 0.6267 - val_acc: 0.6476
Epoch 9/20
 - 0s - loss: 0.5500 - acc: 0.7329 - val_loss: 0.6274 - val_acc: 0.6476
Epoch 10/20
 - 0s - loss: 0.5503 - acc: 0.7310 - val_loss: 0.6205 - val_acc: 0.6476
Epoch 11/20
 - 0s - lo

訓練集的準確度下降了，不過測試集有近7成的準度